<a target="_blank" href="https://colab.research.google.com/github/lukebarousse/Int_SQL_Data_Analytics_Course/blob/main/5_Views/1_View_Intro.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Views Intro

## Overview

### 🥅 Analysis Goals

- What we’re going to use for this dataset to do X e.g. Use the following in order to explore a dataset on experience and salaries
    - Major topic 1
    - Major topic 2
    - Major topic 3
- The end goal of this is e.g. Identify which jobs meet our expectations of years experience and total salary.

### 📘 Concepts Covered

General concepts we’re going to cover

- Concept 1
- Concept 2
- Concept 3

---
## Major Topic

### 📝 Notes

- Add in specific notes

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Problem Description

**`FUNCTION` / Concept Covered**

1. Create a view of this query.

In [ ]:
SELECT 
    EXTRACT(YEAR FROM MIN(orderdate)) AS cohort_year,
    customerkey,
    SUM(quantity * netprice * exchangerate) AS total_customer_net_revenue
FROM sales
GROUP BY 
    customerkey

2. Our original query from the last example.

In [ ]:
%%sql

WITH cohort_analysis AS (
    SELECT 
        EXTRACT(YEAR FROM MIN(orderdate)) AS cohort_year,
        orderdate,
        SUM(quantity * netprice * exchangerate) AS total_net_revenue
    FROM sales
    GROUP BY 
        orderdate
),

rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS user_count,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_user_count
    FROM cohort_analysis
)

SELECT
    cohort_year,
    orderdate,
    cumulative_revenue,
    user_count,
    cumulative_revenue / user_count AS rolling_avg_ltv,
    rolling_7_day_revenue / rolling_7_day_user_count AS rolling_7_avg_ltv
FROM rolling_ltv;

3. Create view in pgadmin

4. Use the view.

In [ ]:
%%sql

WITH rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS user_count,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_user_count
    FROM cohort_analysis
)

SELECT
    cohort_year,
    orderdate,
    cumulative_revenue,
    user_count,
    cumulative_revenue / user_count AS rolling_avg_ltv,
    rolling_7_day_revenue / rolling_7_day_user_count AS rolling_7_avg_ltv
FROM rolling_ltv;